# Splitting Data into Test-Train-Validation Sets

In [17]:
import csv
import argparse
import pandas as pd
import numpy as np
import os

from sklearn.preprocessing import OneHotEncoder

In [56]:
batting_data_path = 'data/Batting.csv'
# INFO:
# 101,332 Players with up to 20 features each (exluding year, including team)
# if metric not reported for player, set to 0.0 by default

df = pd.read_csv(batting_data_path, index_col = 0)
df = df.fillna(0)

,yearID,stint,teamID,lgID,G,AB,R,H,2B,3B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
playerID,,,,,,,,,,,,,,,,,,,,,
abercda01,1871,1,TRO,0,1,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
addybo01,1871,1,RC1,0,25,118.0,30.0,32.0,6.0,0.0,...,13.0,8.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
allisar01,1871,1,CL1,0,29,137.0,28.0,40.0,4.0,5.0,...,19.0,3.0,1.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0
allisdo01,1871,1,WS3,0,27,133.0,28.0,44.0,10.0,2.0,...,27.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
ansonca01,1871,1,RC1,0,25,120.0,29.0,39.0,11.0,3.0,...,16.0,6.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zitoba01,2015,1,OAK,AL,3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zobribe01,2015,1,OAK,AL,67,235.0,39.0,63.0,20.0,2.0,...,33.0,1.0,1.0,33.0,26.0,2.0,0.0,0.0,3.0,5.0
zobribe01,2015,2,KCA,AL,59,232.0,37.0,66.0,16.0,1.0,...,23.0,2.0,3.0,29.0,30.0,1.0,1.0,0.0,2.0,3.0


In [53]:
df_recent_players = df[df.yearID >= 1975] #48k players
team_set = set(df_recent_players.teamID)
df_recent_players

#  NOTE: Potentially create one-hot encoding of teams...
## I chose to leave it off for now because I think it may
## create Data leakage in our model


,yearID,stint,teamID,lgID,G,AB,R,H,2B,3B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
playerID,,,,,,,,,,,,,,,,,,,,,
aaronha01,1975,1,ML4,AL,137,465.0,45.0,109.0,16.0,2.0,...,60.0,0.0,1.0,70.0,51.0,3.0,1.0,1.0,6.0,15.0
abbotgl01,1975,1,OAK,AL,30,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
acostcy01,1975,1,PHI,NL,6,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
adamsgl01,1975,1,SFN,NL,61,90.0,10.0,27.0,2.0,1.0,...,15.0,1.0,0.0,11.0,25.0,0.0,1.0,0.0,1.0,1.0
alburvi01,1975,1,MIN,AL,33,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zitoba01,2015,1,OAK,AL,3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zobribe01,2015,1,OAK,AL,67,235.0,39.0,63.0,20.0,2.0,...,33.0,1.0,1.0,33.0,26.0,2.0,0.0,0.0,3.0,5.0
zobribe01,2015,2,KCA,AL,59,232.0,37.0,66.0,16.0,1.0,...,23.0,2.0,3.0,29.0,30.0,1.0,1.0,0.0,2.0,3.0


In [67]:
#  Randomly shuffle by player year
## NOTE: This may be an inaccurate assumption to make.
## We're implicitly assuming that training on players from 1975
## won't impact prediction accuracy for modern day players.

## If this is an incorrect assumption, consider data augmentation,
## taking into account differences between current-day MLB players

df_recent_players = df.sample(frac=1) #randomly permute by year
player_ids = [df.iloc[:,0:0]]
player_ids[0][0]
# player_ids = set(df_recent_players.playerID) #List of all player ids

# len(player_ids)

KeyError: 0

In [51]:
x_data = df_recent_players[df_recent_players.columns[6:]]